In [18]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as np

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\SUBHAYAN\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [19]:
##loading the dataset
data = gutenberg.raw('shakespeare-hamlet.txt')
## save
with open('hamlet.txt','w') as file:
    file.write(data)

In [20]:
## data preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [21]:
##load the dataset
with open('hamlet.txt','r') as file:
    text = file.read().lower()

##Tokenize the text
tokenizer =Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
total_words

4818

In [22]:
## create input sequences
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [23]:
##pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
print(max_sequence_len)

14


In [24]:
input_sequences = np.array(pad_sequences(input_sequences,maxlen= max_sequence_len,padding = 'pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]],
      shape=(25732, 14), dtype=int32)

In [25]:
##create predictors and label
import tensorflow as tf
#all the words to x, last word to y
x,y  = input_sequences[:,:-1],input_sequences[:,-1]


In [26]:
print(x)
print(len(x),len(x[0]))

[[   0    0    0 ...    0    0    1]
 [   0    0    0 ...    0    1  687]
 [   0    0    0 ...    1  687    4]
 ...
 [   0    0    0 ...  687    4   45]
 [   0    0    0 ...    4   45 1047]
 [   0    0    0 ...   45 1047    4]]
25732 13


In [27]:
print(y)
print(len(y))


[ 687    4   45 ... 1047    4  193]
25732


In [28]:
import tensorflow as tf
x,y = input_sequences[:,:-1],input_sequences[:,-1]

In [29]:
## do one hot encoding 
y = tf.keras.utils.to_categorical(y,num_classes = total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], shape=(25732, 4818))

In [30]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

In [31]:
##early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=7,
    min_delta=0.001,      # stop only if improvement < 0.001
    restore_best_weights=True
)


In [32]:
##train lstm rnn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout

## model defintion
model = Sequential()
model.add(Embedding(total_words,100,))
model.add(LSTM(150,return_sequences  = True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation="softmax"))

##Compile
model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [33]:
## train 
history = model.fit(x_train,y_train,epochs = 100, validation_data= (x_test,y_test),verbose=1)

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - accuracy: 0.0322 - loss: 6.8838 - val_accuracy: 0.0326 - val_loss: 6.7378
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0393 - loss: 6.4512 - val_accuracy: 0.0408 - val_loss: 6.8170
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.0474 - loss: 6.3107 - val_accuracy: 0.0451 - val_loss: 6.8592
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.0525 - loss: 6.1779 - val_accuracy: 0.0464 - val_loss: 6.9063
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.0574 - loss: 6.0422 - val_accuracy: 0.0511 - val_loss: 6.9134
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0654 - loss: 5.8920 - val_accuracy: 0.0581 - val_loss: 6.9542
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 12s 18ms/step - accuracy: 0.0744 - loss: 5.7331 - val_accuracy: 0.0596 - val_loss: 6.9750
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0837 - loss: 5

In [34]:
#predict the next word
def predict_next_word(model,tokenizer,text,max_sequence_len):

    token_list = tokenizer.texts_to_sequences([text])[0]

    if len(token_list)>=max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]
    
    token_list = pad_sequences([token_list],maxlen = max_sequence_len-1,padding = 'pre')
    predicted = model.predict(token_list,verbose = 0)
    predicted_word_index = np.argmax(predicted,axis =1)
    for word,index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [36]:
input_text = "To be or not to"
print(f"Input text: {input_text}")
max_sequence_len = model.input_shape[1]+1
next_word = predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next word predicted:{next_word}")

Input text: To be or not to
Next word predicted:be


In [39]:
import pickle
##Save the model
model.save("next_word_lstm.keras")
#save tokenizer
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [55]:
input_text = "What we two nights"
print(f"Input text: {input_text}")
max_sequence_len = model.input_shape[1]+1
next_word = predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next word predicted:{next_word}")

Input text: What we two nights
Next word predicted:haue
